In [1]:
import pandas as pd
import glob
from pathlib import Path
from collections import defaultdict

def analyze_dataframe_indices():
    # Get all parquet files recursively, excluding ml_datasets
    parquet_files = []
    for pattern in ['../data/ticker_features/*.parquet', '../data/market_features/**/*.parquet']:
        parquet_files.extend(glob.glob(pattern, recursive=True))
    
    # Dictionary to store unique index patterns
    index_patterns = defaultdict(list)
    
    print(f"Analyzing {len(parquet_files)} files...\n")
    
    for file_path in parquet_files:
        try:
            # Read the parquet file
            df = pd.read_parquet(file_path)
            
            # Get index information
            index_type = type(df.index).__name__
            index_dtype = str(df.index.dtype) if hasattr(df.index, 'dtype') else 'N/A'
            index_freq = df.index.freq if hasattr(df.index, 'freq') else 'N/A'
            
            # Create a pattern key
            pattern_key = f"Type: {index_type}, Dtype: {index_dtype}, Freq: {index_freq}"
            
            # Store file info under this pattern
            index_patterns[pattern_key].append({
                'file': Path(file_path).name,
                'range': f"{df.index.min()} to {df.index.max()}" if len(df.index) > 0 else "Empty",
                'length': len(df.index)
            })
            
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
    
    # Print results
    print("Index Pattern Analysis:")
    print("=" * 80)
    
    for pattern, files in index_patterns.items():
        print(f"\nPattern: {pattern}")
        print(f"Number of files with this pattern: {len(files)}")
        
        # Get date ranges
        if 'DatetimeIndex' in pattern:
            min_dates = [pd.to_datetime(f['range'].split(' to ')[0]) for f in files]
            max_dates = [pd.to_datetime(f['range'].split(' to ')[1]) for f in files]
            common_start = max(min_dates)
            common_end = min(max_dates)
            print(f"Common date range: {common_start} to {common_end}")
        
        # Get length statistics
        lengths = [f['length'] for f in files]
        print(f"Row count stats - Min: {min(lengths)}, Max: {max(lengths)}, Mean: {sum(lengths)/len(lengths):.1f}")
        
        # Show a few example files
        print("\nExample files:")
        for file_info in files[:3]:  # Show first 3 examples
            print(f"  - {file_info['file']}")
        if len(files) > 3:
            print(f"  ... and {len(files)-3} more")
        print("-" * 80)

# Run the analysis
analyze_dataframe_indices()

Analyzing 519 files...

Index Pattern Analysis:

Pattern: Type: DatetimeIndex, Dtype: datetime64[ns], Freq: None
Number of files with this pattern: 518
Common date range: 2024-07-08 00:00:00 to 2025-06-04 00:00:00
Row count stats - Min: 230, Max: 1826, Mean: 1248.0

Example files:
  - HON_features.parquet
  - HUBB_features.parquet
  - FDS_features.parquet
  ... and 515 more
--------------------------------------------------------------------------------

Pattern: Type: RangeIndex, Dtype: int64, Freq: N/A
Number of files with this pattern: 1
Row count stats - Min: 503, Max: 503, Mean: 503.0

Example files:
  - sector_mapping.parquet
--------------------------------------------------------------------------------


In [2]:
import pandas as pd
import glob
from pathlib import Path

def find_first_complete_date():
    # Get all parquet files recursively, excluding ml_datasets
    parquet_files = []
    for pattern in ['../data/ticker_features/*.parquet', '../data/market_features/**/*.parquet']:
        parquet_files.extend(glob.glob(pattern, recursive=True))
    
    # Dictionary to store first complete date for each file
    first_complete_dates = {}
    
    for file_path in parquet_files:
        try:
            # Read the parquet file
            df = pd.read_parquet(file_path)
            
            # Ensure the index is datetime
            if not isinstance(df.index, pd.DatetimeIndex):
                print(f"Warning: {file_path} does not have a datetime index")
                continue
            
            # For ticker_features files, exclude analyst_targets column
            if 'ticker_features' in file_path:
                df = df.drop(columns=['analyst_targets'], errors='ignore')
            
            # Find first row without any NaNs
            complete_rows = df.notna().all(axis=1)
            if complete_rows.any():
                first_complete_date = df.index[complete_rows].min()
                first_complete_dates[file_path] = first_complete_date
            else:
                print(f"Warning: {file_path} has no complete rows")
                
        except Exception as e:
            print(f"Error processing {file_path}: {str(e)}")
    
    if not first_complete_dates:
        return None
    
    # Find the latest first complete date across all files
    latest_first_date = max(first_complete_dates.values())
    
    # Print results
    print("\nFirst complete date for each file:")
    for file_path, date in sorted(first_complete_dates.items()):
        print(f"{Path(file_path).name}: {date}")
    
    print(f"\nLatest first complete date across all files: {latest_first_date}")
    return latest_first_date

# Run the function
first_complete_date = find_first_complete_date()


First complete date for each file:
daily_breadth.parquet: 2020-06-04 00:00:00
gdelt_raw.parquet: 2020-06-04 00:00:00
market_sentiment.parquet: 2020-06-23 00:00:00
market_volatility.parquet: 2020-07-30 00:00:00
Basic Materials.parquet: 2020-07-02 00:00:00
Communication Services.parquet: 2020-07-02 00:00:00
Consumer Cyclical.parquet: 2020-07-02 00:00:00
Consumer Defensive.parquet: 2020-07-02 00:00:00
Energy.parquet: 2020-07-02 00:00:00
Financial Services.parquet: 2020-07-02 00:00:00
Healthcare.parquet: 2020-07-02 00:00:00
Industrials.parquet: 2020-07-02 00:00:00
Real Estate.parquet: 2020-07-02 00:00:00
Technology.parquet: 2020-07-02 00:00:00
Utilities.parquet: 2020-07-02 00:00:00
AAPL_features.parquet: 2021-03-19 00:00:00
ABBV_features.parquet: 2021-03-19 00:00:00
ABNB_features.parquet: 2021-09-27 00:00:00
ABT_features.parquet: 2021-03-19 00:00:00
ACGL_features.parquet: 2021-03-19 00:00:00
ACN_features.parquet: 2021-03-19 00:00:00
ADBE_features.parquet: 2021-03-19 00:00:00
ADI_features.